# Gaussian Quadrature: Optimal Numerical Integration

## Introduction

Gaussian quadrature is a powerful numerical integration technique that achieves optimal accuracy for polynomial integrands. Unlike Newton-Cotes methods (trapezoidal rule, Simpson's rule) which use equally-spaced points, Gaussian quadrature strategically chooses both the **nodes** (evaluation points) and **weights** to maximize precision.

## Theoretical Foundation

### Basic Formulation

A quadrature rule approximates a definite integral as a weighted sum:

$$\int_a^b f(x) \, dx \approx \sum_{i=1}^{n} w_i f(x_i)$$

where $x_i$ are the **nodes** and $w_i$ are the **weights**.

### Gaussian Quadrature Theorem

For Gauss-Legendre quadrature on the interval $[-1, 1]$:

$$\int_{-1}^{1} f(x) \, dx \approx \sum_{i=1}^{n} w_i f(x_i)$$

The nodes $x_i$ are the roots of the **Legendre polynomial** $P_n(x)$, and the weights are:

$$w_i = \frac{2}{(1 - x_i^2)[P_n'(x_i)]^2}$$

### Key Property: Exactness

An $n$-point Gaussian quadrature rule is **exact** for all polynomials of degree up to $2n - 1$. This is optimal—no other $n$-point rule can achieve higher precision.

### Legendre Polynomials

The Legendre polynomials satisfy the recurrence relation:

$$(n+1)P_{n+1}(x) = (2n+1)xP_n(x) - nP_{n-1}(x)$$

with $P_0(x) = 1$ and $P_1(x) = x$.

### Change of Interval

To integrate over $[a, b]$ instead of $[-1, 1]$, use the transformation:

$$\int_a^b f(x) \, dx = \frac{b-a}{2} \int_{-1}^{1} f\left(\frac{b-a}{2}t + \frac{a+b}{2}\right) dt$$

## Error Analysis

For a function $f \in C^{2n}[a, b]$, the error of $n$-point Gaussian quadrature is:

$$E_n[f] = \frac{(b-a)^{2n+1}(n!)^4}{(2n+1)[(2n)!]^3} f^{(2n)}(\xi)$$

for some $\xi \in (a, b)$. The error decreases rapidly as $n$ increases.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import roots_legendre
from scipy import integrate

# Set up plotting style
plt.rcParams['figure.figsize'] = [12, 10]
plt.rcParams['font.size'] = 11

## Implementation

### Computing Gauss-Legendre Nodes and Weights

We'll use SciPy's `roots_legendre` function, which computes the nodes (roots of Legendre polynomials) and corresponding weights efficiently.

In [ ]:
def gauss_legendre_quadrature(f, a, b, n):
    """
    Compute the integral of f from a to b using n-point Gauss-Legendre quadrature.
    
    Parameters:
    -----------
    f : callable
        Function to integrate
    a, b : float
        Integration limits
    n : int
        Number of quadrature points
    
    Returns:
    --------
    float
        Approximate value of the integral
    """
    # Get nodes and weights for [-1, 1]
    nodes, weights = roots_legendre(n)
    
    # Transform nodes from [-1, 1] to [a, b]
    transformed_nodes = 0.5 * (b - a) * nodes + 0.5 * (a + b)
    
    # Apply quadrature formula with Jacobian factor
    integral = 0.5 * (b - a) * np.sum(weights * f(transformed_nodes))
    
    return integral


def visualize_nodes_weights(n_points):
    """
    Visualize the Gauss-Legendre nodes and weights.
    """
    nodes, weights = roots_legendre(n_points)
    return nodes, weights

## Demonstration: Nodes and Weights Distribution

Let's visualize how Gaussian quadrature nodes and weights are distributed for different numbers of points.

In [ ]:
# Visualize nodes and weights for different n
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

n_values = [3, 5, 7, 10]

for ax, n in zip(axes.flat, n_values):
    nodes, weights = roots_legendre(n)
    
    # Plot nodes with size proportional to weights
    ax.scatter(nodes, np.zeros_like(nodes), s=weights*500, 
               c='steelblue', alpha=0.7, edgecolors='navy')
    ax.axhline(y=0, color='gray', linestyle='-', linewidth=0.5)
    
    # Add weight labels
    for xi, wi in zip(nodes, weights):
        ax.annotate(f'{wi:.3f}', (xi, 0.02), ha='center', fontsize=8)
    
    ax.set_xlim(-1.2, 1.2)
    ax.set_ylim(-0.1, 0.15)
    ax.set_xlabel('Node position $x_i$')
    ax.set_title(f'Gauss-Legendre: n = {n} points\nDegree of exactness: {2*n-1}')
    ax.set_yticks([])

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("Nodes and weights saved to plot.png")

## Convergence Analysis

Let's compare the convergence of Gaussian quadrature with Simpson's rule for several test functions.

In [ ]:
def simpsons_rule(f, a, b, n):
    """
    Composite Simpson's rule with n subintervals (n must be even).
    """
    if n % 2 == 1:
        n += 1
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = f(x)
    
    integral = y[0] + y[-1]
    integral += 4 * np.sum(y[1:-1:2])
    integral += 2 * np.sum(y[2:-1:2])
    
    return integral * h / 3


# Test functions
test_functions = {
    r'$e^x$': (lambda x: np.exp(x), 0, 1, np.exp(1) - 1),
    r'$\sin(x)$': (lambda x: np.sin(x), 0, np.pi, 2.0),
    r'$\frac{1}{1+x^2}$': (lambda x: 1/(1+x**2), 0, 1, np.pi/4),
    r'$\sqrt{x}$': (lambda x: np.sqrt(x), 0, 1, 2/3),
}

# Compute errors
n_range = np.arange(2, 21)

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

for ax, (name, (f, a, b, exact)) in zip(axes.flat, test_functions.items()):
    gauss_errors = []
    simpson_errors = []
    
    for n in n_range:
        # Gaussian quadrature error
        gauss_result = gauss_legendre_quadrature(f, a, b, n)
        gauss_errors.append(abs(gauss_result - exact))
        
        # Simpson's rule error (using 2n function evaluations for fair comparison)
        simpson_result = simpsons_rule(f, a, b, 2*n)
        simpson_errors.append(abs(simpson_result - exact))
    
    ax.semilogy(n_range, gauss_errors, 'o-', label='Gaussian Quadrature', 
                color='steelblue', markersize=5)
    ax.semilogy(n_range, simpson_errors, 's--', label="Simpson's Rule",
                color='coral', markersize=5)
    
    ax.set_xlabel('Number of points n')
    ax.set_ylabel('Absolute error')
    ax.set_title(f'Integrand: {name}')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_ylim(bottom=1e-16)

plt.tight_layout()
plt.show()

## Exactness for Polynomials

A fundamental property: $n$-point Gaussian quadrature is exact for polynomials up to degree $2n-1$.

In [ ]:
def test_polynomial_exactness():
    """
    Demonstrate that n-point Gaussian quadrature is exact for polynomials
    of degree up to 2n-1.
    """
    print("Testing Polynomial Exactness Property")
    print("=" * 50)
    
    for n in [2, 3, 4, 5]:
        print(f"\n{n}-point Gaussian quadrature (exact for degree ≤ {2*n-1}):")
        
        for degree in range(1, 2*n + 2):
            # Test polynomial x^degree on [0, 1]
            f = lambda x, d=degree: x**d
            exact = 1 / (degree + 1)  # Exact integral of x^d from 0 to 1
            
            approx = gauss_legendre_quadrature(f, 0, 1, n)
            error = abs(approx - exact)
            
            status = "✓ EXACT" if error < 1e-14 else f"✗ Error: {error:.2e}"
            print(f"  Degree {degree:2d}: {status}")

test_polynomial_exactness()

## Practical Application: Computing Difficult Integrals

Gaussian quadrature excels at integrals that are challenging for equally-spaced methods.

In [ ]:
# Example: Integral with oscillatory behavior
def oscillatory_integral():
    """
    Compute: ∫₀¹ sin(10πx) e^(-x) dx
    """
    f = lambda x: np.sin(10 * np.pi * x) * np.exp(-x)
    
    # Reference value from scipy
    exact, _ = integrate.quad(f, 0, 1)
    
    print("Oscillatory Integral: ∫₀¹ sin(10πx) e^(-x) dx")
    print("=" * 50)
    print(f"Reference value: {exact:.12f}\n")
    
    print("Gaussian Quadrature Convergence:")
    for n in [5, 10, 15, 20, 25, 30]:
        result = gauss_legendre_quadrature(f, 0, 1, n)
        error = abs(result - exact)
        print(f"  n = {n:2d}: {result:>15.12f}  (error: {error:.2e})")

oscillatory_integral()

In [ ]:
# Example: Gaussian integral (probability)
def gaussian_probability():
    """
    Compute the probability P(-1 ≤ Z ≤ 1) for standard normal distribution.
    ∫_{-1}^{1} (1/√(2π)) e^(-x²/2) dx
    """
    f = lambda x: (1 / np.sqrt(2 * np.pi)) * np.exp(-x**2 / 2)
    
    # Reference: this equals erf(1/√2) ≈ 0.6826894921
    from scipy.special import erf
    exact = erf(1 / np.sqrt(2))
    
    print("\nGaussian Probability: P(-1 ≤ Z ≤ 1)")
    print("=" * 50)
    print(f"Reference value: {exact:.12f}\n")
    
    for n in [3, 5, 7, 10]:
        result = gauss_legendre_quadrature(f, -1, 1, n)
        error = abs(result - exact)
        print(f"  n = {n:2d}: {result:.12f}  (error: {error:.2e})")

gaussian_probability()

## Summary

### Key Takeaways

1. **Optimal Efficiency**: $n$-point Gaussian quadrature is exact for polynomials of degree $2n-1$, which is the theoretical maximum.

2. **Rapid Convergence**: For smooth functions, the error decreases exponentially with $n$, vastly outperforming Newton-Cotes methods.

3. **Non-uniform Nodes**: The nodes cluster near the endpoints, which helps control Runge's phenomenon.

4. **Orthogonal Polynomials**: The nodes are roots of Legendre polynomials, connecting quadrature to the rich theory of orthogonal polynomials.

### When to Use Gaussian Quadrature

- **Ideal for**: Smooth integrands, high precision requirements, computational efficiency
- **Less suitable for**: Functions with singularities at nodes, highly oscillatory functions requiring adaptive methods

### Extensions

Other Gaussian quadrature variants handle special cases:
- **Gauss-Chebyshev**: For integrands with $\sqrt{1-x^2}$ weight function
- **Gauss-Laguerre**: For semi-infinite intervals $[0, \infty)$ with $e^{-x}$ weight
- **Gauss-Hermite**: For infinite intervals $(-\infty, \infty)$ with $e^{-x^2}$ weight